## **Step 1: Please select a GPU runtime**

![](https://i.imgur.com/RUIixAQ.png) ![image.png](https://i.imgur.com/zKc74pP.png)

## **Step 2: Install TVM by running the following block.**

We have pre-compiled a tvm build for your convenience.

![](https://i.imgur.com/k9U5WCB.png)

In [8]:
# Let's first install TVM!
# This TVM git tag is: 3b8715df7ea9263b71e30888f1aa112bd8cfcfdc
# which is prior to our 1st detected bug.
!pip install wget
import os
import wget

# Your Python should be Python 3.7
pyversion = os.popen('python3 --version').read()
print('Your python version is ', pyversion)

def install_tvm(pyv: int):
    return wget.download(
        'https://github.com/Tzer-AnonBot/tzer/releases/download/tvm-0.8.dev1040/' +
        'tlcpack_nightly-0.8.dev1040+g3b8715df7-cp{}-cp{}'.format(pyv, pyv) +
        'm-manylinux_2_17_x86_64.manylinux2014_x86_64.whl')

whl_name = None
if '3.8' in pyversion:
    # TVM for Python 3.8
    whl_name = install_tvm(38)
elif '3.7' in pyversion: 
    # TVM for Python 3.7
    whl_name = install_tvm(37)
elif '3.6' in pyversion:
    # TVM for Python 3.6
    whl_name = install_tvm(36)
else:
    print('Please make sure you have Python 3.6,7,8. Actually, 3.7+ is recommended.')

if whl_name:
    os.system('python3 -m pip install ' + whl_name)
    import tvm
    print('Successfully installed TVM!')
else:
    print('Failed to install tvm...')

Your python version is  Python 3.7.12



## **Step 3: Click the bottoms to run bugs!**

![](https://i.imgur.com/OG7YPlK.png)

## Notes: Bug symptom

For bugs whose symptom is crash (e.g., Bug 1), you will see:

![](https://i.imgur.com/iRuFd1H.png)

## Bug 1

In [ ]:
# Crash. You will see "Your session crashed for an unknown reason" after running this bug.
import tvm
from tvm import tir

v = tir.Cast('bool', tvm.runtime.convert("a"))
body = tir.stmt.While(v, body=tir.Evaluate(tir.const(0)))
func = tir.PrimFunc(params={}, body=body)
mod = tvm.lower(func)
nopt_mod = tvm.build(mod)

## Bug 2 & Bug 3

In [ ]:
import tvm
from tvm import ir, tir

a = tir.Var("a", "int32")
b = tir.Var("b", "handle")
iter_var = tir.IterVar(ir.Range(0,1 ), a, 1)
buffer = tir.buffer.decl_buffer((1,))
buffer_map = {b: buffer}
store = tir.Store(buffer.data, tir.const(1), tir.const(1))
attr_stmt = tir.AttrStmt(iter_var, "coproc_uop_scope", tir.const(1), store)
f = tir.PrimFunc({a, b}, body=attr_stmt, buffer_map=buffer_map)
mod = tvm.lower(f)
tvm.build(mod)

In [ ]:
import tvm
from tvm import ir, tir

a = tir.Var("a", "int32")
b = tir.Var("b", "handle")
iter_var = tir.IterVar(ir.Range(0,1 ), a, 1)
buffer = tir.buffer.decl_buffer((1,))
buffer_map = {b: buffer}
store = tir.Store(buffer.data, tir.const(1), tir.const(1))
attr_stmt = tir.AttrStmt(iter_var, "compute_scope", tir.const(1), store)
f = tir.PrimFunc({a, b}, body=attr_stmt, buffer_map=buffer_map)
mod = tvm.lower(f)
tvm.build(mod)

## Bug 4

In [ ]:
import tvm
print(tvm.tir.Shuffle([1],[1]).dtype)

AttributeError: ignored

## Bug 5 & Bug 6 & Bug 7 & Bug 8

In [ ]:
import tvm

analyzer = tvm.arith.Analyzer()
analyzer.rewrite_simplify(tvm.tir.Div(tvm.tir.Ramp(1,1,2), tvm.tir.Broadcast(0, 2)))

In [ ]:
import tvm

analyzer = tvm.arith.Analyzer()
analyzer.rewrite_simplify(tvm.tir.Mod(tvm.tir.Ramp(1,1,2), tvm.tir.Broadcast(0, 2)))

In [ ]:
import tvm

analyzer = tvm.arith.Analyzer()
analyzer.rewrite_simplify(tvm.tir.FloorDiv(tvm.tir.Ramp(1,1,2), tvm.tir.Broadcast(0, 2)))

In [ ]:
import tvm

analyzer = tvm.arith.Analyzer()
analyzer.rewrite_simplify(tvm.tir.FloorMod(tvm.tir.Ramp(1,1,2), tvm.tir.Broadcast(0, 2)))

## Bug 9

In [ ]:
import tvm

from tvm import relay
from tvm.relay.testing import create_workload

simple_net = relay.nn.conv2d(
    data=relay.var("data", relay.TensorType((1, 3, 224, 224), "float32")),
    weight=relay.var("weight"),
    kernel_size=(5, 5),
    channels=3,
    padding=(1, 1),
)
simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)
mod, _ = create_workload(simple_net)

old_mod = mod

with tvm.transform.PassContext(opt_level=4):
    with tvm.target.Target("llvm"):
        seq = tvm.transform.Sequential(passes=[relay.transform.ToBasicBlockNormalForm()], opt_level=4)
        new_mod = seq(mod)

assert old_mod.astext() == mod.astext()
assert old_mod.astext() != new_mod.astext()

AssertionError: ignored

## Bug 10

In [ ]:
from tvm import tir
hash(tir.StringImm("s"))

TypeError: ignored

## Bug 11

In [ ]:
import tvm
import numpy as np
from tvm import tir, te

n = te.size_var("n")
m = te.size_var("m")
A = te.placeholder((n, n), name="A", dtype="int32")

T = te.compute((m, m), lambda i, j: A[i][j])
s = te.create_schedule(T.op)
ir_m = tvm.lower(s, [A, T])


inputs = [tvm.nd.array(np.random.uniform(0, 100, size=(32, 32)).astype("int32"))]
output = tvm.nd.empty((32, 32), "int32")

with tvm.transform.PassContext(opt_level=4):
    opt = tvm.transform.Sequential(
        [tir.transform.DecorateDeviceScope()]
    )
    mod = opt(ir_m)

opt_execute = tvm.build(mod, [*inputs, output], tvm.target.Target("llvm"))
opt_execute(*[inputs[0], output])

## Bug 12

In [ ]:
import tvm
from tvm import tir

tvm.build(tir.PrimFunc([], tir.Evaluate(tir.ret(tir.const(0)))))

TVMError: ignored

## Bug 13

In [ ]:
from tvm import tir
tir.Var(name=1, dtype='int')

## Bug 14

In [ ]:
from tvm import tir
print({tir.const(1), tir.const(True)})

ValueError: ignored

## Bug 15 & Bug 16

In [ ]:
from tvm import tir
import tvm

zero = tir.const(0)
nop = tir.Evaluate(zero)
v = tir.Var("i1", "int32")
for_stmt = tir.For(v, zero, zero, tir.ForKind.SERIAL, nop)
load = tir.Evaluate(tir.Load("int32", v, zero))
seq = tir.SeqStmt([for_stmt, for_stmt, load])
func = tir.PrimFunc([], seq)
mod = tvm.IRModule({"main": func})
mod = tir.transform.InjectVirtualThread()(
    mod
)  # Use pass InjectVirtualThread to invoke ConvertSSA


In [ ]:
from tvm import tir
import tvm

zero = tir.const(0)
nop = tir.Evaluate(zero)
v = tir.Var("i1", "int32")
for_stmt = tir.For(v, zero, zero, tir.ForKind.SERIAL, nop)
store = tir.Store(v, zero, zero)
seq = tir.SeqStmt([for_stmt, for_stmt, store])
func = tir.PrimFunc([], seq)
mod = tvm.IRModule({"main": func})
mod = tir.transform.InjectVirtualThread()(
    mod
)  # Use pass InjectVirtualThread to invoke ConvertSSA




## Bug 17 & Bug 18

In [ ]:
import tvm
array = tvm.runtime.convert([1, 2, 3])
print(array.type_key)
print(array.test_key)

Array


TypeError: ignored

In [ ]:
import tvm
from tvm import te
a = te.var("a")
b = te.var("b")
amap = tvm.runtime.convert({a: 2, b: 3})
print(amap.type_key)
print(amap.test_key)

Map


TypeError: ignored

## Bug 19

In [ ]:
import tvm
from tvm import tir

var = tir.Var('a',dtype='int32')
buf = tir.decl_buffer((1,), name='buf')
buf_load = tir.expr.BufferLoad(buffer=buf, indices=tvm.runtime.convert([0]))
buf_load_stmt = tir.stmt.Evaluate(buf_load)
for_loop = tir.stmt.For(loop_var=var, kind=1, min_val=1, extent=buf_load, body=buf_load_stmt)
buf_func = tir.PrimFunc(params={}, body=for_loop)

tvm.lower(buf_func)

## Bug 20 & Bug 21 & Bug 22

In [ ]:
# API Misuse in 3 of previous tutorials (bring_your_own_datatypes.py, from_keras.py, from_onnx.py )
# We only show one motivating example here.

import tvm
import tvm.relay as relay
from tvm.relay import testing
from tvm import IRModule
import time

shape = (1, 3, 100, 100)

def example():
    return testing.squeezenet.get_workload(batch_size=1, num_classes=100, image_shape=shape[1:], dtype='float32')
    data = relay.var("data", relay.TensorType(shape, "float32"))
    weight = relay.var("weight")
    bn_gamma = relay.var("bn_gamma")
    bn_beta = relay.var("bn_beta")
    bn_mmean = relay.var("bn_mean")
    bn_mvar = relay.var("bn_var")

    simple_net = relay.nn.conv2d(
        data=data, weight=weight, kernel_size=(5, 5), channels=32, padding=(1, 1)
    )
    simple_net = relay.nn.batch_norm(simple_net, bn_gamma, bn_beta, bn_mmean, bn_mvar)[0]
    simple_net = relay.nn.relu(simple_net)
    simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)

    return testing.create_workload(simple_net)

def error_usage(): # Used in previous tutorial
    mod, params = example()
    target = tvm.target.Target('llvm')
    dev = tvm.cpu()
    with tvm.transform.PassContext(opt_level=4): 
        executor = relay.build_module.create_executor("vm", mod, dev, target)
    evaluated = executor.evaluate()

    t0 = time.time()
    tvm_out = evaluated(
        tvm.nd.empty(shape=shape, 
        device=dev, 
        dtype='float32'), **params)
    print(f'Elapsed time by API-Misuse case: {time.time() - t0}')

def good_usage(): # After correction.
    mod, params = example()
    target = tvm.target.Target('llvm')
    dev = tvm.cpu()
    with tvm.transform.PassContext(opt_level=4): 
        mod = IRModule.from_expr(relay.build_module.bind_params_by_name(mod["main"], params))
        executor = relay.build_module.create_executor("vm", mod, dev, target).evaluate()
    t0 = time.time()
    tvm_out = executor(
        tvm.nd.empty(shape=shape, 
        device=dev, 
        dtype='float32'), **params)
    print(f'Elapsed time by correct case: {time.time() - t0}')

if __name__ == '__main__':
    error_usage()
    good_usage()

Elapsed time by API-Misuse case: 0.22153639793395996
Elapsed time by correct case: 0.18967914581298828


## Bug 23

In [ ]:
# Imcompatible passes (but actually independent) introduced by inconsistency.

import tvm
import tvm.testing

from tvm import relay
from tvm.relay import testing

data = relay.var("data", relay.TensorType((1, 3, 64, 64), "float32"))
weight = relay.var("weight")

bn_gamma = relay.var("bn_gamma")
bn_beta = relay.var("bn_beta")
bn_mmean = relay.var("bn_mean")
bn_mvar = relay.var("bn_var")

simple_net = relay.nn.conv2d(
    data=data, weight=weight, kernel_size=(3, 3), channels=3, padding=(1, 1)
)
simple_net = relay.nn.batch_norm(simple_net, bn_gamma, bn_beta, bn_mmean, bn_mvar)[0]
simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)

module, params = testing.create_workload(simple_net)

# Apply some simple passes to legalize the IR.
with tvm.transform.PassContext(opt_level=0):
    module, params = relay.optimize(module, tvm.testing.enabled_targets()[0][0], params)

seq = tvm.transform.Sequential([relay.transform.AnnotateSpans(), relay.transform.DefuseOps()])
with tvm.transform.PassContext(opt_level=3):
    module = seq(module)

...100%, 0.47 MB, 1596 KB/s, 0 seconds passed


TVMError: ignored

## Bug 24 & Bug 25 & Bug 26

In [ ]:
import tvm
tvm.tir.expr.Call(None, None, None, None)

In [ ]:
import tvm
tvm.tir.generic.add(None, None)

In [ ]:
import tvm
tvm.tir.stmt.Allocate(None, None, None, None, None, None)

## Bug 27

In [ ]:
# CuDNN context error. The script ended with a hang and segfault. 
# This bug is reproducible on CentOS 7. Other platform might not be able to reproduce this.


!uname -a # Linux of Google Colab is not CentOS 7 and cannot reproduce this bug.

# The output of this program on CentOS 7
"""
[03:09:31] ../src/runtime/contrib/cudnn/conv_forward.cc:245:    CUDNN Found 8 fwd algorithms, choosing CUDNN_CONVOLUTION_FWD_ALGO_IMPLICIT_GEMM
[03:09:31] ../src/runtime/contrib/cudnn/conv_forward.cc:248:            0) CUDNN_CONVOLUTION_FWD_ALGO_IMPLICIT_GEMM - time: 0.06144 ms, Memory: 0
[03:09:31] ../src/runtime/contrib/cudnn/conv_forward.cc:248:            1) CUDNN_CONVOLUTION_FWD_ALGO_IMPLICIT_PRECOMP_GEMM - time: 0.104448 ms, Memory: 304000
[03:09:31] ../src/runtime/contrib/cudnn/conv_forward.cc:248:            2) CUDNN_CONVOLUTION_FWD_ALGO_GEMM - time: 0.110592 ms, Memory: 5419008
[03:09:31] ../src/runtime/contrib/cudnn/conv_forward.cc:248:            3) CUDNN_CONVOLUTION_FWD_ALGO_WINOGRAD - time: 0.146432 ms, Memory: 18176
[03:09:31] ../src/runtime/contrib/cudnn/conv_forward.cc:248:            4) CUDNN_CONVOLUTION_FWD_ALGO_FFT - time: 0.916384 ms, Memory: 26949312
[03:09:31] ../src/runtime/contrib/cudnn/conv_forward.cc:248:            5) CUDNN_CONVOLUTION_FWD_ALGO_FFT_TILING - time: 1.10106 ms, Memory: 374272
[03:09:31] ../src/runtime/contrib/cudnn/conv_forward.cc:248:            6) CUDNN_CONVOLUTION_FWD_ALGO_WINOGRAD_NONFUSED - time: 1.79712 ms, Memory: 137288448
[03:09:31] ../src/runtime/contrib/cudnn/conv_forward.cc:248:            7) CUDNN_CONVOLUTION_FWD_ALGO_DIRECT - time: -1 ms, Memory: 0
One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
[1]    134780 segmentation fault (core dumped)  python3 test.py
""" 

content = """
import tvm
import tvm.relay as relay
import numpy as np
from tvm.relay import testing

def example():
    out_channels = 16
    batch_size = 1

    data = relay.var("data", relay.TensorType((batch_size, 3, 224, 224), "float32"))
    weight = relay.var("weight")
    bn_gamma = relay.var("bn_gamma")
    bn_beta = relay.var("bn_beta")
    bn_mmean = relay.var("bn_mean")
    bn_mvar = relay.var("bn_var")

    simple_net = relay.nn.conv2d(
        data=data, weight=weight, kernel_size=(3, 3), channels=out_channels, padding=(1, 1)
    )
    simple_net = relay.nn.batch_norm(simple_net, bn_gamma, bn_beta, bn_mmean, bn_mvar)[0]
    simple_net = relay.nn.relu(simple_net)
    simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)

    return testing.create_workload(simple_net)

def func():
    data = np.zeros((1, 3, 224, 224))
    mod, params = example()
    target = tvm.target.Target('cuda -libs=cudnn')
    dev = tvm.cuda()
    with tvm.transform.PassContext(opt_level=3):
        executor = relay.build_module.create_executor("graph", mod, dev, target)

    tvm_out = executor.evaluate()(tvm.nd.array(data.astype('float32')), **params)
"""

with open('test.py', 'w') as f:
    f.write(content)

!python3 test.py

Linux 26e8f5ac1921 5.4.104+ #1 SMP Sat Jun 5 09:50:34 PDT 2021 x86_64 x86_64 x86_64 GNU/Linux


## Bug 28 & 29



In [ ]:
# There are 2 bugs here. 
# 1. One is about OOM.
# 2. Another is the incorrect exception. (The exception should be OOM not "device type = 0")

import tvm
import tvm.relay as relay
import numpy as np
from tvm.relay import testing

def example():
    out_channels = 32

    data = relay.var("data", relay.TensorType((relay.Any(), 3, 224, 224), "float32"))
    weight = relay.var("weight")
    bn_gamma = relay.var("bn_gamma")
    bn_beta = relay.var("bn_beta")
    bn_mmean = relay.var("bn_mean")
    bn_mvar = relay.var("bn_var")

    simple_net = relay.nn.conv2d(
        data=data, weight=weight, kernel_size=(3, 3), channels=out_channels, padding=(1, 1)
    )
    simple_net = relay.nn.batch_norm(simple_net, bn_gamma, bn_beta, bn_mmean, bn_mvar)[0]
    simple_net = relay.nn.relu(simple_net)
    simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)

    return testing.create_workload(simple_net)

if __name__ == '__main__':
    mod, params = example()
    # compile the model
    target = tvm.target.Target('cuda')
    dev = tvm.cuda()
    with tvm.transform.PassContext(opt_level=3):
        executor = relay.build_module.create_executor("vm", mod, dev, target)

    for i in range(100):
        print(f'Running batch size = {i}')  # Should be OOM error, but a later exception received.
        tvm_out = executor.evaluate()(tvm.nd.empty(shape=(i, 3, 224, 224), device=dev, dtype='float32'), **params)

One or more operators have not been tuned. Please tune your model for better performance. Use DEBUG logging level to see more details.
Cannot find config for target=cuda -keys=cuda,gpu -max_num_threads=1024 -thread_warp_size=32, workload=('conv2d_nchw.cuda', ('TENSOR', (any_dim, 3, 224, 224), 'float32'), ('TENSOR', (32, 3, 3, 3), 'float32'), (1, 1), (1, 1, 1, 1), (1, 1), 'float32'). A fallback configuration is used, which may bring great performance regression.


Running batch size = 0
Running batch size = 1
Running batch size = 2
Running batch size = 3
Running batch size = 4
Running batch size = 5
Running batch size = 6
Running batch size = 7
Running batch size = 8
Running batch size = 9
Running batch size = 10
Running batch size = 11
Running batch size = 12
Running batch size = 13
Running batch size = 14
Running batch size = 15
Running batch size = 16
Running batch size = 17
Running batch size = 18
Running batch size = 19
Running batch size = 20
Running batch size = 21
Running batch size = 22
Running batch size = 23
Running batch size = 24
Running batch size = 25
Running batch size = 26
Running batch size = 27
Running batch size = 28
Running batch size = 29
Running batch size = 30
Running batch size = 31
Running batch size = 32
Running batch size = 33
Running batch size = 34
Running batch size = 35
Running batch size = 36
Running batch size = 37
Running batch size = 38
Running batch size = 39
Running batch size = 40
Running batch size = 41
Ru

## Bug 30

In [ ]:
# We tested tvm across various runtimes (vm, graph, debug, etc.)
# We found debug module fails for 0-batch input (viable for other runtime types)
# We observed CUDA internal error using cuda-gdb.

import tvm
import tvm.relay as relay
import numpy as np
from tvm.relay import testing

def example():
    data = relay.var("data", relay.TensorType((relay.Any(), 3, 128, 128), "float32"))
    simple_net = relay.nn.conv2d(
        data=data, weight=relay.var("weight"), kernel_size=(3, 3), channels=8, padding=(1, 1)
    )
    simple_net = relay.Function(relay.analysis.free_vars(simple_net), simple_net)

    return testing.create_workload(simple_net)

if __name__ == '__main__':
    data = np.zeros((0, 3, 128, 128))
    mod, params = example()
    target = tvm.target.Target('cuda')
    dev = tvm.cuda()
    with tvm.transform.PassContext(opt_level=2):
        executor = relay.build_module.create_executor("debug", mod, dev, target)

    tvm_out = executor.evaluate()(tvm.nd.array(data.astype('float32')), **params)

Cannot find config for target=cuda -keys=cuda,gpu -max_num_threads=1024 -thread_warp_size=32, workload=('conv2d_nchw.cuda', ('TENSOR', (any_dim, 3, 128, 128), 'float32'), ('TENSOR', (8, 3, 3, 3), 'float32'), (1, 1), (1, 1, 1, 1), (1, 1), 'float32'). A fallback configuration is used, which may bring great performance regression.


## Bug 31

In [ ]:
from tvm import tir
import tvm

a = tir.Broadcast(tir.const(1), 2)
v = tir.Var('i1', 'int32')
stmt = tir.Store(v, a, a, None)
func = tir.PrimFunc([v], stmt)
tvm.build(func)

## Bug 32

In [ ]:
import tvm
buf = tvm.tir.buffer.decl_buffer((1,))
value = tvm.tir.IntImm('int32', 1)
i = tvm.tir.IntImm('int32x1', 1)
index = tvm.tir.Shuffle([i, i], [i])
s = tvm.tir.Store(buf.data, value, index)
f = tvm.tir.PrimFunc({buf.data}, s)
tvm.build(f)

## Bug 33

In [ ]:
import tvm

v = tvm.tir.Var('v', 'float32')
value = tvm.tir.isnan(v)
op = value.op

buf = tvm.tir.buffer.decl_buffer((1,))
value = tvm.tir.Call('int32', op, [0])
s = tvm.tir.Store(buf.data, value, 0)
f = tvm.tir.PrimFunc({buf.data}, s)
tvm.build(f)

## Bug 34

In [ ]:
import tvm
from tvm import tir

var = tir.Var(name='v', dtype='int32')
buf = tir.decl_buffer((1,), name='buf')
buf_load = tir.expr.BufferLoad(buffer=buf, indices=tvm.runtime.convert([0]))
then_case = tir.Store(buffer_var=var,value=buf_load,index=tvm.runtime.convert(0))
for_body = then_case
for_stmt = tir.For(loop_var=var, min_val=0, extent=0, kind=1,body=for_body)
y = tir.IfThenElse(then_case=then_case,else_case=for_stmt,condition=tvm.runtime.convert(False))
f=tir.PrimFunc(body=y,params=[var])

mod = tvm.IRModule({'main':f})
mod = tir.transform.PlanAndUpdateBufferAllocationLocation()(mod)
mod = tir.transform.CompactBufferAllocation()(mod)
mod = tir.transform.LowerMatchBuffer()(mod)

## Bug 35

In [ ]:
import tvm
from tvm import tir
# import os
# print(os.getpid())
# input()
v = tir.Broadcast(0, 8)
index = tir.Ramp(72,1,8)
buf = tir.buffer.decl_buffer((1, 0))
store = tir.Store(buf.data, v, index)
loop_var = tir.Var('v', 'int32')
for_loop = tir.For(loop_var, 0, 4, tir.ForKind.VECTORIZED, store)
f = tir.PrimFunc({buf.data}, for_loop)
tvm.build(f)

ModuleNotFoundError: ignored

## Bug 36

In [ ]:
# This bug happens to Debug Mode
import tvm
from tvm import ir, tir

a = tir.Var("a", "int32")
iter_var = tir.IterVar(ir.Range(0,1 ), a, 1, "vthread")
attr_stmt = tir.AttrStmt(iter_var, "virtual_thread",tir.op.floormod(tir.ret(tir.IntImm('int32',0)), 3), tir.Evaluate(tir.const(0)))
f = tir.PrimFunc({a}, body=attr_stmt)#, buffer_map=buffer_map)
mod = tvm.lower(f)
tvm.build(mod)

## Bug 37

In [ ]:
import tvm
from tvm import tir
s_v = tir.Var('buf', 'handle')
buf = tir.buffer.decl_buffer((1, 0))
store = tir.Store(buf.data, tir.IntImm('int32', 0), 0, tvm.runtime.convert(32))
f = tir.PrimFunc({s_v}, store, buffer_map={s_v:buf})
tvm.build(f)

## Bug 38

In [ ]:
import tvm
from tvm import tir

v = tir.Var('v', 'int32')
s = tir.Select(tir.Cast('bool',v), 150, 1)
let_stmt = tir.LetStmt(v, s, tir.Evaluate(v))
f = tir.PrimFunc({v},let_stmt)
tvm.build(f)

## Bug 39

In [ ]:
import tvm
from tvm import tir
expr = tir.Div(tir.FloorDiv(tir.Cast('int32', 2.95148e+28), tir.atan(tir.IntImm('int32',1))), tir.FloorMod(303, 32))
stmt = tir.Evaluate(expr)
func = tir.PrimFunc({},stmt)
tvm.build(func)

## Bug 40

In [ ]:
import tvm
from tvm import tir

var = tir.Var('var', 'int32')
false_value = tir.Cast('int32', tir.acosh(tir.Cast('float32', var)))
value = tir.Select(var > 0, tir.const(0), false_value)
let_stmt = tir.LetStmt(var, value, tir.Evaluate(var))
f = tir.PrimFunc({}, let_stmt)
tvm.build(f)